In [1]:
import asyncio
import datetime as dt
import math
from typing import Literal

import matplotlib.pyplot as plt
import mplfinance as mpf
import numpy as np
import pandas as pd
import pandas_market_calendars as mcal
import plotly.graph_objects as go
import polars as pl
from dash import Dash, dcc, html
from plotly.subplots import make_subplots

nse = mcal.get_calendar("NSE")

In [2]:
pd.set_option("display.max_rows", 25_000)
pd.set_option("display.max_columns", 500)
pl.Config.set_tbl_cols(500)
pl.Config.set_tbl_rows(10_000)

pd.options.display.float_format = "{:.4f}".format

In [3]:
import sys

sys.path.append("..")
from tooling.enums import AssetClass, Index, Spot, StrikeSpread
from tooling.fetch import fetch_option_data, fetch_spot_data
from tooling.filter import find_atm, option_tool

In [4]:
from fetching_from_local_db.enums import AssetClass, Index, StrikeSpread
from fetching_from_local_db.fetch_from_db import _fetch_batch, fetch_data, fetch_spot_data

In [5]:
from expiries import dict_expiries

In [6]:
def resample(
    data: pl.DataFrame, timeframe, offset: dt.timedelta | None = None
) -> pl.DataFrame:
    return (
        data.set_sorted("datetime")
        .group_by_dynamic(
            index_column="datetime",
            every=timeframe,
            period=timeframe,
            label="left",
            offset=offset,
        )
        .agg(
            [
                pl.col("o").first().alias("o"),
                pl.col("h").max().alias("h"),
                pl.col("l").min().alias("l"),
                pl.col("c").last().alias("c"),
                # pl.col("volume").sum().alias("volume"),
            ]
        )
    )


# ohlc_resampled = resample(pl.DataFrame(bnf_1min), '7d', pd.Timedelta(days=4))
# ohlc_resampled

In [117]:
bnf_1min = pd.read_csv("../data/nifty_min.csv")

In [118]:
bnf_1min.columns = ['index', 'datetime', 'o', 'h', 'l', 'c', 'v']
bnf_1min.tail()

,index,datetime,o,h,l,c,v
743635,nifty,2025-02-28 15:25:00,22110.4500,22119.0500,22109.2500,22118.8000,0
743636,nifty,2025-02-28 15:26:00,22118.6000,22123.8500,22113.2000,22114.7000,0
743637,nifty,2025-02-28 15:27:00,22114.0000,22120.1500,22108.4500,22114.1500,0
743638,nifty,2025-02-28 15:28:00,22114.1500,22121.2500,22106.1000,22113.8500,0
743639,nifty,2025-02-28 15:29:00,22112.5500,22127.4000,22109.3500,22127.4000,0


In [119]:
bnf_1min["datetime"] = pd.to_datetime(bnf_1min["datetime"]).dt.tz_localize(None)
# bnf_1min = bnf_1min[((bnf_1min['datetime'].dt.year == 2020) & (bnf_1min['datetime'].dt.month == 4))]
bnf_1min = bnf_1min[
    (bnf_1min["datetime"].dt.year >= 2019) & (bnf_1min["datetime"].dt.year <= 2025)
]

In [120]:
dict_expiries

{'nifty': [datetime.datetime(2017, 1, 25, 0, 0),
  datetime.datetime(2017, 2, 23, 0, 0),
  datetime.datetime(2017, 3, 30, 0, 0),
  datetime.datetime(2017, 4, 27, 0, 0),
  datetime.datetime(2017, 5, 25, 0, 0),
  datetime.datetime(2017, 6, 29, 0, 0),
  datetime.datetime(2017, 7, 27, 0, 0),
  datetime.datetime(2017, 8, 31, 0, 0),
  datetime.datetime(2017, 9, 28, 0, 0),
  datetime.datetime(2017, 10, 26, 0, 0),
  datetime.datetime(2017, 11, 30, 0, 0),
  datetime.datetime(2017, 12, 28, 0, 0),
  datetime.datetime(2018, 1, 25, 0, 0),
  datetime.datetime(2018, 2, 22, 0, 0),
  datetime.datetime(2018, 3, 28, 0, 0),
  datetime.datetime(2018, 4, 26, 0, 0),
  datetime.datetime(2018, 5, 31, 0, 0),
  datetime.datetime(2018, 6, 28, 0, 0),
  datetime.datetime(2018, 7, 26, 0, 0),
  datetime.datetime(2018, 8, 30, 0, 0),
  datetime.datetime(2018, 9, 27, 0, 0),
  datetime.datetime(2018, 10, 25, 0, 0),
  datetime.datetime(2018, 11, 29, 0, 0),
  datetime.datetime(2018, 12, 27, 0, 0),
  datetime.datetime(2019,

In [11]:
from datetime import date
from bisect import bisect_right

def get_next_expiry(input_date, index_symbol):
    expiries = dict_expiries.get(index_symbol)
    if not expiries:
        return None
        
    expiry_dates = sorted({dt.date() for dt in expiries})
    pos = bisect_right(expiry_dates, input_date.date())
    return expiry_dates[pos] if pos < len(expiry_dates) else None


In [12]:
index_ = 'nifty'

if index_ == 'nifty':
    LOT_SIZE_ = 75
    STRIKE_SPREAD_ = 50
    INDEX_LEVERAGE_ = 8

In [132]:
import pandas as pd
import numpy as np

def calculate_weekly_levels(df, x_percent):
    # Convert datetime to Pandas datetime type
    df['datetime'] = pd.to_datetime(df['datetime'])

    # Set datetime as index
    df = df.set_index('datetime')

    # Define week from Friday to Thursday
    df['Week'] = (df.index - pd.DateOffset(days=1)).to_period('W-THU')

    # Resample to get weekly OHLC (Friday Open - Thursday Close)
    weekly_df = df.resample('W-THU').agg({'open': 'first', 'high': 'max', 'low': 'min', 'close': 'last'})

    # Calculate weekly range and previous week's range
    weekly_df['Range'] = weekly_df['high'] - weekly_df['low']
    weekly_df['Prev_Week_Range'] = weekly_df['Range'].shift(1)

    return weekly_df

async def execute_trading_strategy(df, x_percent, tf, offset):
    
    df = df.rename(columns={'o': 'open', 'h': 'high', 'l': 'low', 'c': 'close'})
    weekly_levels = calculate_weekly_levels(df, x_percent)
    weekly_levels = weekly_levels[:-1]
    # print(weekly_levels.head().to_string())
    tradebook = []

    # Iterate over all weeks
    for week_till, row in weekly_levels.iterrows():
        # print("Previous Week Till : ", week_till)
        if pd.isna(row['Prev_Week_Range']):  
            continue  # Skip the first week as we need the previous week's range

        # Get current week's Friday open
        current_friday_date = week_till + dt.timedelta(days=1)
        # print("Current Friday : ", current_friday_date.date())
        current_friday_open = df.loc[df['datetime'].dt.date >= current_friday_date.date(), 'open'].iloc[0]

        # Dynamic multiplier
        # multiplier = 1 * np.sqrt(row['Range'] / current_friday_open)
        multiplier = x_percent * (row['Range'] / current_friday_open)
        if multiplier*100 > 10*x_percent:            # To avoid any trade with previous week range > 10% of current market open
            continue
        # print(multiplier)
        # Calculate high & low levels based on previous week's range
        # high_level = current_friday_open + (x_percent / 100) * row['Range']
        # low_level = current_friday_open - (x_percent / 100) * row['Range']

        high_level = current_friday_open + multiplier * row['Range']      # HL = 23000 + (0.21 * 500) = 23105
        low_level = current_friday_open - multiplier * row['Range']

        # Round levels to nearest 50
        high_level_strike = np.round(high_level / STRIKE_SPREAD_) * STRIKE_SPREAD_
        low_level_strike = np.round(low_level / STRIKE_SPREAD_) * STRIKE_SPREAD_

        # print(current_friday_open, high_level, high_level_strike, low_level, low_level_strike)

        # Initialize trade activity flags
        pe_trade_active = True
        ce_trade_active = True

        current_week_start = current_friday_date
        current_week_end = current_friday_date + dt.timedelta(days=6, hours=15, minutes=30)
        
        # Filter the DataFrame for the current week's data
        current_week_data = df[(df['datetime'] >= current_week_start) & (df['datetime'] <= current_week_end)]
        # print(current_week_data.to_string())

        expiry = get_next_expiry(current_week_start, index_)
        dte = (expiry - current_week_start.date()).days
        if dte >= 7:
            continue
        ce_df = await fetch_data(
            index=index_,
            expiry=expiry,
            strike=int(high_level_strike),
            asset_class='C',
            start_date=current_week_start.date(),
            start_time=dt.time(9, 16),
            end_date=expiry,
            end_time=dt.time(15, 30),
        )
        if not isinstance(ce_df, str) and ce_df is not None:
            ce_df = resample(ce_df, tf, offset)
            ce_df_pandas = ce_df.to_pandas()
            ce_entry_price = ce_df_pandas.iloc[0]['o']
        else:
            ce_entry_price = float('nan')

        pe_df = await fetch_data(
            index=index_,
            expiry=expiry,
            strike=int(low_level_strike),
            asset_class='P',
            start_date=current_week_start.date(),
            start_time=dt.time(9, 16),
            end_date=expiry,
            end_time=dt.time(15, 30),
        )
        if not isinstance(pe_df, str) and pe_df is not None:
            pe_df = resample(pe_df, tf, offset)
            pe_df_pandas = pe_df.to_pandas()
            pe_entry_price = pe_df_pandas.iloc[0]['o']
        else:
            pe_entry_price = float('nan')
        
        tradebook.append({
            'week_start': current_week_start,
            'entry_time': dt.datetime.combine(current_week_start, dt.time(9, 16)),
            'current week open': current_friday_open,
            'previous week high': row['high'],
            'previous week low': row['low'],
            'previous week range': row['Range'],
            'multiplier%': multiplier*100,
            'high level': high_level, 
            'low level': low_level,
            'strike': int(high_level_strike),
            'type': 'CE',
            'expiry': expiry,
            "tag": "STRANGLE",
            # 'action': 'SELL'
            'entry price': ce_entry_price,
        })
        tradebook.append({
            'week_start': current_week_start,
            'entry_time': dt.datetime.combine(current_week_start, dt.time(9, 16)),
            'current week open': current_friday_open,
            'previous week high': row['high'],
            'previous week low': row['low'],
            'previous week range': row['Range'],
            'multiplier%': multiplier*100,
            'high level': high_level, 
            'low level': low_level,
            'strike': int(low_level_strike),
            'type': 'PE',
            'expiry': expiry,
            "tag": "STRANGLE",
            # 'action': 'SELL'
            'entry price': pe_entry_price,
        })
        # print('CE Entry', ce_entry_price, '\nPE Entry', pe_entry_price)

        # Manage trades during the week
        # print(current_week_data.to_string())
        for i in range(0, len(current_week_data)):
            dte = (expiry - (current_week_data['datetime'].iloc[i]).date()).days
            # prev_close = current_week_data['close'].iloc[i - 1]
            curr_close = current_week_data['close'].iloc[i]
            curr_datetime = current_week_data['datetime'].iloc[i]
            # print(curr_datetime, curr_close, high_level, low_level)
            # Square off if breached
            if ce_trade_active and curr_close >= high_level:
                for trade in tradebook:
                    if (
                        trade['type'] == 'CE' and 
                        trade['week_start'] == current_week_start and 
                        'exit_time' not in trade
                    ):
                        df_row = ce_df_pandas[ce_df_pandas['datetime'] <= curr_datetime]
                        ce_exit_price = df_row['c'].iloc[-1] if len(df_row) != 0 else float('nan')
                        trade['exit price'] = ce_exit_price
                        trade['exit_time'] = curr_datetime
                        # print('CE Exit', ce_exit_price)
                        break
                ce_trade_active = False  # Mark CE trade as inactive
        
            if pe_trade_active and curr_close <= low_level:
                for trade in tradebook:
                    if (
                        trade['type'] == 'PE' and
                        trade['week_start'] == current_week_start and 
                        'exit_time' not in trade
                    ):
                        df_row = pe_df_pandas[pe_df_pandas['datetime'] <= curr_datetime]
                        pe_exit_price = df_row['c'].iloc[-1] if len(df_row) != 0 else float('nan')
                        trade['exit price'] = pe_exit_price
                        trade['exit_time'] = curr_datetime
                        # print('PE Exit', pe_exit_price)
                        break
                pe_trade_active = False  # Mark PE trade as inactive
        
            # Re-enter if closed back inside range
            if not ce_trade_active and curr_close <= high_level and dte > 0:
                df_row = ce_df_pandas[ce_df_pandas['datetime'] >= curr_datetime]
                ce_entry_price = df_row['c'].iloc[0] if len(df_row) != 0 else float('nan')
                # print('CE Entry', ce_entry_price)
                tradebook.append(
                    {
                        'week_start': current_week_start,
                        'entry_time': curr_datetime,
                        'current week open': current_friday_open,
                        'previous week high': row['high'],
                        'previous week low': row['low'],
                        'previous week range': row['Range'],
                        'multiplier%': multiplier*100,
                        'high level': high_level,
                        'low level': low_level,
                        'strike': int(high_level_strike),
                        'type': 'CE',
                        'expiry': expiry,
                        "tag": "RE-ENTRY",
                        # 'action': 'SELL'
                        'entry price': ce_entry_price,
                    }
                )
                ce_trade_active = True  # Reactivate CE trade
        
            if not pe_trade_active and curr_close >= low_level and dte > 0:
                df_row = pe_df_pandas[pe_df_pandas['datetime'] >= curr_datetime]
                pe_entry_price = df_row['c'].iloc[0] if len(df_row) != 0 else float('nan')
                # print('PE Entry', pe_entry_price)
                tradebook.append(
                    {
                        'week_start': current_week_start,
                        'entry_time': curr_datetime,
                        'current week open': current_friday_open,
                        'previous week high': row['high'],
                        'previous week low': row['low'],
                        'previous week range': row['Range'],
                        'multiplier%': multiplier*100,
                        'high level': high_level,
                        'low level': low_level,
                        'strike': int(low_level_strike),
                        'type': 'PE',
                        'expiry': expiry,
                        "tag": "RE-ENTRY",
                        # 'action': 'SELL'
                        'entry price': pe_entry_price,
                    }
                )
                pe_trade_active = True  # Reactivate PE trade

            if curr_datetime == current_week_data.iloc[-1]['datetime']:
                for trade in tradebook:
                    if (
                        trade['week_start'] == current_week_start and
                        'exit_time' not in trade
                    ):
                        trade['exit_time'] = curr_datetime
                        
                        if trade['type'] == 'CE':
                            trade['exit price'] = ce_df_pandas['c'].iloc[-1]
                            # print('CE Exit', ce_df_pandas['c'].iloc[-1])
                        else:
                            trade['exit price'] = pe_df_pandas['c'].iloc[-1]
                            # print('PE Exit', pe_df_pandas['c'].iloc[-1])

    tb = pd.DataFrame(tradebook)
    tb['entry_time'] = pd.to_datetime(tb['entry_time'])
    tb['expiry'] = pd.to_datetime(tb['expiry'])
    tb['dte'] = (tb['expiry'].dt.date - tb['entry_time'].dt.date).apply(lambda x: x.days)
    tb['slippage'] = 0.01 * (tb['entry price'] + tb['exit price'])
    tb['final points'] = tb['entry price'] - tb['exit price'] - tb['slippage']
    tb['portfolio'] = 1_00_00_000
    tb['index leverage'] = np.where(tb['type'] == 'PE', 8, 9)
    tb['qty'] = tb['portfolio'] * tb['index leverage'] / tb['strike'] 
    tb['PnL'] = tb['qty'] * tb['final points']
    tb['ROI%'] = tb['PnL'] * 100 / tb['portfolio']
    tb['Trade Year'] = tb['week_start'].dt.year
    
    return tb


In [133]:
tf = '15m'
offset = '0m'
bnf_1h = resample(pl.DataFrame(bnf_1min), tf, offset)
tb = await execute_trading_strategy(bnf_1h.to_pandas(), 5, tf, offset)

In [134]:
# tb.to_csv('RBW_W_Strangle.csv', index=False)

In [135]:
# tb['slippage'] = 0.01 * (tb['entry price'] + tb['exit price'])
# tb['final points'] = tb['entry price'] - tb['exit price'] - tb['slippage']
# tb['portfolio'] = 1_00_00_000
# tb['index leverage'] = 8
# tb['qty'] = tb['portfolio'] * tb['index leverage'] / tb['strike']
# tb['PnL'] = tb['qty'] * tb['final points']
# tb['ROI%'] = tb['PnL'] * 100 / tb['portfolio']
# tb['Trade Year'] = tb['week_start'].dt.year

In [136]:
def generate_stats(tb_expiry, ema_window):
    stats_df8 = pd.DataFrame(
        index=range(2019, 2026),
        columns=[
            "Total ROI",
            "Total Trades",
            "Win Rate",
            "Avg Profit% per Trade",
            "Avg Loss% per Trade",
            "Max Drawdown",
            "ROI/DD Ratio",
            "Variation",
        ],
    )
    combined_df_sorted = tb_expiry
    # combined_df_sorted = tb_expiry_ce
    # combined_df_sorted = tb_expiry_pe

    # Iterate over each year
    for year in range(2019, 2026):
        # Filter trades for the current year
        year_trades = combined_df_sorted[(combined_df_sorted["Trade Year"] == year)]

        # Calculate total ROI
        total_roi = year_trades["ROI%"].sum()

        # Calculate total number of trades
        total_trades = len(year_trades)

        # Calculate win rate
        win_rate = (year_trades["ROI%"] > 0).mean() * 100

        # Calculate average profit per trade
        avg_profit = year_trades[year_trades["ROI%"] > 0]["ROI%"].mean()

        # Calculate average loss per trade
        avg_loss = year_trades[year_trades["ROI%"] < 0]["ROI%"].mean()

        # Calculate maximum drawdown
        max_drawdown = (
            year_trades["ROI%"].cumsum() - year_trades["ROI%"].cumsum().cummax()
        ).min()

        # Calculate ROI/DD ratio
        roi_dd_ratio = total_roi / abs(max_drawdown)

        variation = f"{ema_window}"

        # Store the statistics in the DataFrame
        stats_df8.loc[year] = [
            total_roi,
            total_trades,
            win_rate,
            avg_profit,
            avg_loss,
            max_drawdown,
            roi_dd_ratio,
            variation,
        ]

    # Calculate overall statistics
    overall_total_roi = stats_df8["Total ROI"].sum()
    overall_total_trades = stats_df8["Total Trades"].sum()
    overall_win_rate = (combined_df_sorted["ROI%"] > 0).mean() * 100
    overall_avg_profit = combined_df_sorted[combined_df_sorted["ROI%"] > 0][
        "ROI%"
    ].mean()
    overall_avg_loss = combined_df_sorted[combined_df_sorted["ROI%"] < 0]["ROI%"].mean()
    overall_max_drawdown = (
        combined_df_sorted["ROI%"].cumsum()
        - combined_df_sorted["ROI%"].cumsum().cummax()
    ).min()
    overall_roi_dd_ratio = overall_total_roi / abs(overall_max_drawdown)
    overall_variation = variation

    # Store the overall statistics in the DataFrame
    stats_df8.loc["Overall"] = [
        overall_total_roi,
        overall_total_trades,
        overall_win_rate,
        overall_avg_profit,
        overall_avg_loss,
        overall_max_drawdown,
        overall_roi_dd_ratio,
        overall_variation,
    ]
    return {overall_roi_dd_ratio: stats_df8}

In [137]:
tb[tb['Trade Year'] == 2020]

,week_start,entry_time,current week open,previous week high,previous week low,previous week range,multiplier%,high level,low level,strike,type,expiry,tag,entry price,exit price,exit_time,dte,slippage,final points,portfolio,index leverage,qty,PnL,ROI%,Trade Year
215,2020-01-03,2020-01-03 09:16:00,12261.1000,12289.7500,12152.2500,137.5000,5.6072,12268.8099,12253.3901,12250,CE,2020-01-09,STRANGLE,61.6000,0.0500,2020-01-09 15:15:00,6,0.6165,60.9335,10000000,9,7346.9388,447674.6939,4.4767,2020
216,2020-01-03,2020-01-03 09:16:00,12261.1000,12289.7500,12152.2500,137.5000,5.6072,12268.8099,12253.3901,12250,PE,2020-01-09,STRANGLE,65.8500,65.0000,2020-01-03 09:15:00,6,1.3085,-0.4585,10000000,8,6530.6122,-2994.2857,-0.0299,2020
217,2020-01-03,2020-01-03 10:00:00,12261.1000,12289.7500,12152.2500,137.5000,5.6072,12268.8099,12253.3901,12250,PE,2020-01-09,RE-ENTRY,70.9000,72.0000,2020-01-03 10:15:00,6,1.4290,-2.5290,10000000,8,6530.6122,-16515.9184,-0.1652,2020
218,2020-01-10,2020-01-10 09:16:00,12271.0000,12265.5000,11931.3500,334.1500,13.6154,12316.4960,12225.5040,12300,CE,2020-01-16,STRANGLE,50.7000,81.9500,2020-01-13 09:15:00,6,1.3265,-32.5765,10000000,9,7317.0732,-238364.6341,-2.3836,2020
219,2020-01-10,2020-01-10 09:16:00,12271.0000,12265.5000,11931.3500,334.1500,13.6154,12316.4960,12225.5040,12250,PE,2020-01-16,STRANGLE,75.8000,0.0500,2020-01-16 15:15:00,6,0.7585,74.9915,10000000,8,6530.6122,489740.4082,4.8974,2020
220,2020-01-10,2020-01-13 13:00:00,12271.0000,12265.5000,11931.3500,334.1500,13.6154,12316.4960,12225.5040,12300,CE,2020-01-16,RE-ENTRY,66.8500,69.3000,2020-01-13 14:30:00,3,1.3615,-3.8115,10000000,9,7317.0732,-27889.0244,-0.2789,2020
221,2020-01-10,2020-01-15 09:15:00,12271.0000,12265.5000,11931.3500,334.1500,13.6154,12316.4960,12225.5040,12300,CE,2020-01-16,RE-ENTRY,42.3000,67.3000,2020-01-15 15:00:00,1,1.0960,-26.0960,10000000,9,7317.0732,-190946.3415,-1.9095,2020
222,2020-01-17,2020-01-17 09:16:00,12328.4000,12389.0000,12213.6000,175.4000,7.1137,12340.8774,12315.9226,12350,CE,2020-01-23,STRANGLE,48.9000,62.8500,2020-01-17 09:15:00,6,1.1175,-15.0675,10000000,9,7287.4494,-109803.6437,-1.0980,2020
223,2020-01-17,2020-01-17 09:16:00,12328.4000,12389.0000,12213.6000,175.4000,7.1137,12340.8774,12315.9226,12300,PE,2020-01-23,STRANGLE,57.1000,52.7500,2020-01-20 10:00:00,6,1.0985,3.2515,10000000,8,6504.0650,21147.9675,0.2115,2020
224,2020-01-17,2020-01-20 09:45:00,12328.4000,12389.0000,12213.6000,175.4000,7.1137,12340.8774,12315.9226,12350,CE,2020-01-23,RE-ENTRY,47.1000,0.0500,2020-01-23 15:15:00,3,0.4715,46.5785,10000000,9,7287.4494,339438.4615,3.3944,2020


In [141]:
stats = generate_stats(tb, 'RBWS')
for x, y in stats.items():
    z = pd.DataFrame(y)
    break

z

,Total ROI,Total Trades,Win Rate,Avg Profit% per Trade,Avg Loss% per Trade,Max Drawdown,ROI/DD Ratio,Variation
2019,66.1813,215,34.8837,2.8947,-1.0937,-16.0332,4.1278,RBWS
2020,88.9280,209,33.0144,4.4622,-1.5640,-18.5099,4.8044,RBWS
2021,107.1351,233,38.6266,3.2251,-1.2896,-18.9536,5.6525,RBWS
2022,53.5406,252,28.9683,4.0004,-1.3474,-21.7900,2.4571,RBWS
2023,52.5310,250,30.8000,2.2973,-0.7447,-8.2897,6.3369,RBWS
2024,45.4697,220,36.3636,2.7411,-1.2781,-19.0696,2.3844,RBWS
2025,11.0613,51,27.4510,3.7463,-1.1186,-15.8756,0.6967,RBWS
Overall,424.8471,1430,33.4266,3.2551,-1.2071,-21.7900,19.4973,RBWS


In [50]:
stats = generate_stats(tb, 'RBW w Strangle')
for x, y in stats.items():
    z = pd.DataFrame(y)
    break

z

,Total ROI,Total Trades,Win Rate,Avg Profit% per Trade,Avg Loss% per Trade,Max Drawdown,ROI/DD Ratio,Variation
2019,62.5858,284,33.4507,2.3193,-0.8435,-16.8432,3.7158,RBW w Strangle
2020,88.4264,267,32.2097,3.6311,-1.2367,-17.2654,5.1216,RBW w Strangle
2021,93.7770,297,34.6801,2.8391,-1.0293,-18.9320,4.9533,RBW w Strangle
2022,53.4370,315,28.5714,3.3433,-1.1097,-17.8082,3.0007,RBW w Strangle
2023,53.5603,312,27.5641,2.0730,-0.5721,-7.8160,6.8527,RBW w Strangle
2024,34.5312,287,32.0557,2.3884,-0.9697,-17.6542,1.9560,RBW w Strangle
2025,-0.9855,74,21.6216,3.1353,-0.8819,-18.3458,-0.0537,RBW w Strangle
Overall,385.3322,1836,30.9368,2.7713,-0.9503,-19.4072,19.8551,RBW w Strangle


In [139]:
tb_ce = tb[tb['type'] == 'CE']
tb_pe = tb[tb['type'] == 'PE']
tb_strangle = tb[tb['tag'] == 'STRANGLE']

In [140]:
stats = generate_stats(tb_strangle, 'STR RBW w Strangle')
for x, y in stats.items():
    z = pd.DataFrame(y)
    break

z

,Total ROI,Total Trades,Win Rate,Avg Profit% per Trade,Avg Loss% per Trade,Max Drawdown,ROI/DD Ratio,Variation
2019,26.2049,96,41.6667,3.1371,-1.8051,-17.6287,1.4865,STR RBW w Strangle
2020,80.0036,94,45.7447,4.6180,-2.3249,-16.6610,4.8018,STR RBW w Strangle
2021,33.6583,106,39.6226,3.7789,-1.9850,-17.1818,1.9590,STR RBW w Strangle
2022,48.2943,104,40.3846,4.4303,-2.2963,-16.9803,2.8441,STR RBW w Strangle
2023,17.2189,104,34.6154,2.3919,-1.0599,-14.3881,1.1967,STR RBW w Strangle
2024,-1.9576,104,37.5000,2.9329,-1.8467,-40.4900,-0.0483,STR RBW w Strangle
2025,19.1521,18,38.8889,4.8075,-1.3182,-8.3771,2.2862,STR RBW w Strangle
Overall,222.5745,626,39.7764,3.6264,-1.8489,-40.4900,5.4970,STR RBW w Strangle


In [92]:
stats = generate_stats(tb_pe, 'PE RBW w Strangle')
for x, y in stats.items():
    z = pd.DataFrame(y)
    break

z

,Total ROI,Total Trades,Win Rate,Avg Profit% per Trade,Avg Loss% per Trade,Max Drawdown,ROI/DD Ratio,Variation
2019,56.8883,133,37.5940,2.3823,-0.7682,-10.7534,5.2903,PE RBW w Strangle
2020,84.1057,126,39.6825,3.9389,-1.4847,-45.8113,1.8359,PE RBW w Strangle
2021,45.1248,161,33.5404,3.1555,-1.1818,-30.1017,1.4991,PE RBW w Strangle
2022,34.3500,175,25.7143,3.7186,-1.0309,-30.2840,1.1343,PE RBW w Strangle
2023,34.7068,167,29.3413,2.0717,-0.5912,-9.9323,3.4944,PE RBW w Strangle
2024,7.1680,147,32.6531,2.2525,-1.0407,-26.3534,0.2720,PE RBW w Strangle
2025,-22.3750,48,14.5833,2.1826,-0.9184,-21.3928,-1.0459,PE RBW w Strangle
Overall,239.9686,957,31.6614,2.9000,-0.9934,-45.8113,5.2382,PE RBW w Strangle


In [59]:
stats = generate_stats(tb, 'RBW w Strangle')
for x, y in stats.items():
    z = pd.DataFrame(y)
    break

z

,Total ROI,Total Trades,Win Rate,Avg Profit% per Trade,Avg Loss% per Trade,Max Drawdown,ROI/DD Ratio,Variation
2019,62.5858,284,33.4507,2.3193,-0.8435,-16.8432,3.7158,RBW w Strangle
2020,88.4264,267,32.2097,3.6311,-1.2367,-17.2654,5.1216,RBW w Strangle
2021,93.7770,297,34.6801,2.8391,-1.0293,-18.9320,4.9533,RBW w Strangle
2022,53.4370,315,28.5714,3.3433,-1.1097,-17.8082,3.0007,RBW w Strangle
2023,53.5603,312,27.5641,2.0730,-0.5721,-7.8160,6.8527,RBW w Strangle
2024,34.5312,287,32.0557,2.3884,-0.9697,-17.6542,1.9560,RBW w Strangle
2025,-0.9855,74,21.6216,3.1353,-0.8819,-18.3458,-0.0537,RBW w Strangle
Overall,385.3322,1836,30.9368,2.7713,-0.9503,-19.4072,19.8551,RBW w Strangle


# TF : 10min , Multiplier : 5 * (prev. week range / current open)
# No trades if previous week range > 10% of current spot price

In [51]:
roi_summary = tb.groupby('dte').agg(
    total_roi_pct=('ROI%', 'sum'),
    trade_count=('ROI%', 'count'),
    win_count=('ROI%', lambda x: (x > 0).sum())
).reset_index()

# Calculate win rate as a percentage
roi_summary['win_rate_pct'] = (roi_summary['win_count'] / roi_summary['trade_count']) * 100

# Optional: drop win_count column if not needed
roi_summary = roi_summary.drop(columns='win_count')
roi_summary

,dte,total_roi_pct,trade_count,win_rate_pct
0,0,9.1325,182,37.3626
1,1,37.6249,197,31.9797
2,2,10.8680,253,22.9249
3,3,30.2776,227,24.6696
4,4,0.1750,1,100.0000
5,5,22.4842,35,40.0000
6,6,274.7699,924,33.3333


In [467]:
stats = generate_stats(tb, 'RBW w Strangle')
for x, y in stats.items():
    z = pd.DataFrame(y)
    break

z

,Total ROI,Total Trades,Win Rate,Avg Profit% per Trade,Avg Loss% per Trade,Max Drawdown,ROI/DD Ratio,Variation
2019,67.1717,284,33.8028,2.3866,-0.8707,-17.0875,3.9311,RBW w Strangle
2020,91.3731,267,31.8352,3.8527,-1.2973,-21.0607,4.3386,RBW w Strangle
2021,100.6335,297,36.0269,2.8830,-1.0997,-20.0095,5.0293,RBW w Strangle
2022,81.9807,315,30.4762,3.3011,-1.0826,-13.7765,5.9508,RBW w Strangle
2023,65.5546,312,31.0897,1.9529,-0.5984,-7.2697,9.0175,RBW w Strangle
2024,47.8182,287,35.5401,2.2067,-0.9634,-16.8124,2.8442,RBW w Strangle
2025,-0.6254,100,26.0000,2.6235,-0.9302,-17.3922,-0.0360,RBW w Strangle
Overall,453.9064,1862,32.7068,2.7335,-0.9772,-21.0607,21.5523,RBW w Strangle


In [468]:
stats = generate_stats(tb, 'RBW w Strangle')
for x, y in stats.items():
    z = pd.DataFrame(y)
    break

z

,Total ROI,Total Trades,Win Rate,Avg Profit% per Trade,Avg Loss% per Trade,Max Drawdown,ROI/DD Ratio,Variation
2019,67.1717,284,33.8028,2.3866,-0.8707,-17.0875,3.9311,RBW w Strangle
2020,91.3731,267,31.8352,3.8527,-1.2973,-21.0607,4.3386,RBW w Strangle
2021,100.6335,297,36.0269,2.8830,-1.0997,-20.0095,5.0293,RBW w Strangle
2022,81.9807,315,30.4762,3.3011,-1.0826,-13.7765,5.9508,RBW w Strangle
2023,65.5546,312,31.0897,1.9529,-0.5984,-7.2697,9.0175,RBW w Strangle
2024,47.8182,287,35.5401,2.2067,-0.9634,-16.8124,2.8442,RBW w Strangle
2025,-0.6254,100,26.0000,2.6235,-0.9302,-17.3922,-0.0360,RBW w Strangle
Overall,453.9064,1862,32.7068,2.7335,-0.9772,-21.0607,21.5523,RBW w Strangle


In [470]:
tb.to_csv("RBW w Strangle 10min.csv", index=False)

In [96]:
stats = generate_stats(tb, 'RBW w Strangle')
for x, y in stats.items():
    z = pd.DataFrame(y)
    break

z

,Total ROI,Total Trades,Win Rate,Avg Profit% per Trade,Avg Loss% per Trade,Max Drawdown,ROI/DD Ratio,Variation
2019,41.4154,132,64.3939,1.5856,-2.0296,-18.2533,2.2689,RBW w Strangle
2020,40.2254,145,62.7586,2.9047,-4.1501,-61.4071,0.6551,RBW w Strangle
2021,46.4850,145,62.7586,1.7585,-2.1835,-18.3801,2.5291,RBW w Strangle
2022,44.4147,139,58.9928,2.1531,-2.4471,-15.9400,2.7864,RBW w Strangle
2023,30.9280,137,62.7737,1.1154,-1.3265,-7.8012,3.9645,RBW w Strangle
2024,22.7406,129,62.7907,1.5129,-2.0792,-13.9160,1.6341,RBW w Strangle
2025,8.9918,37,62.1622,1.5597,-1.9200,-7.6133,1.1811,RBW w Strangle
Overall,235.2010,864,62.3843,1.8368,-2.3812,-65.1730,3.6089,RBW w Strangle


# Original Stats with 50% Range Multiplier and 1hr TF

In [52]:
weekly_roi = tb.groupby('week_start')['ROI%'].sum().reset_index()
weekly_roi['Trade Year'] = weekly_roi['week_start'].dt.year

In [53]:
weekly_roi

,week_start,ROI%,Trade Year
0,2019-01-25,1.2667,2019
1,2019-02-08,2.8490,2019
2,2019-02-15,4.0308,2019
3,2019-02-22,-1.7491,2019
4,2019-03-01,5.1137,2019
5,2019-03-08,2.4936,2019
6,2019-03-15,-4.1937,2019
7,2019-03-22,4.2486,2019
8,2019-03-29,5.6841,2019
9,2019-04-05,-1.5188,2019


In [54]:
stats = generate_stats(weekly_roi, 'RBW w Strangle')
for x, y in stats.items():
    z = pd.DataFrame(y)
    break

z

,Total ROI,Total Trades,Win Rate,Avg Profit% per Trade,Avg Loss% per Trade,Max Drawdown,ROI/DD Ratio,Variation
2019,62.5858,48,75.0000,2.4049,-1.9993,-5.9911,10.4465,RBW w Strangle
2020,88.4264,47,76.5957,3.7897,-4.3638,-10.2018,8.6678,RBW w Strangle
2021,93.7770,53,79.2453,2.9809,-2.8564,-15.2665,6.1427,RBW w Strangle
2022,53.4370,52,69.2308,2.6503,-2.7984,-7.9606,6.7127,RBW w Strangle
2023,53.5603,52,78.8462,1.5787,-1.1166,-2.1245,25.2107,RBW w Strangle
2024,34.5312,52,67.3077,2.2088,-2.5163,-13.8644,2.4906,RBW w Strangle
2025,-0.9855,9,66.6667,2.7848,-5.8980,-9.8775,-0.0998,RBW w Strangle
Overall,385.3322,313,74.1214,2.5964,-2.7472,-15.2665,25.2404,RBW w Strangle


In [124]:
# SIMULATION
multiplier_range = [1, 2, 3, 4, 5]
tf = ['5m', '10m', '15m', '20m', '30m', '45m', '60m']

for i in multiplier_range:
    for j in tf:
        variation = f'Multiplier {i}, TF {j}'
        print(variation)
        
        if j == '10m':
            offset = '5m'
        elif j >= '20m':
            offset = '15m'
        else:
            offset = '0m'
            
        bnf_1h = resample(pl.DataFrame(bnf_1min), j, offset)
        tb = await execute_trading_strategy(bnf_1h.to_pandas(), i, j, offset)

        if len(tb) > 0:
            stats = generate_stats(tb, variation)
            for x, y in stats.items():
                z = pd.DataFrame(y)
                print(z.to_string())


Multiplier 1, TF 5m
        Total ROI Total Trades Win Rate Avg Profit% per Trade Avg Loss% per Trade Max Drawdown ROI/DD Ratio            Variation
2019      52.4642          463  25.0540                2.1285             -0.5702     -24.5867       2.1338  Multiplier 1, TF 5m
2020     100.4390          389  24.6787                3.5159             -0.8092     -22.6950       4.4256  Multiplier 1, TF 5m
2021      26.7037          599  22.7045                2.1169             -0.5654     -28.0479       0.9521  Multiplier 1, TF 5m
2022      86.7835          486  22.6337                2.9741             -0.6568     -17.4074       4.9855  Multiplier 1, TF 5m
2023      -0.4822          501  22.1557                1.5785             -0.4673     -16.8912      -0.0285  Multiplier 1, TF 5m
2024       0.1484          514  23.3463                2.0215             -0.6281     -30.8961       0.0048  Multiplier 1, TF 5m
2025      22.3460           63  23.8095                3.7535             -0.